# Aula 04 - Manipulação Básica de DataFrames

## **Resumo**: Exerícios de manipulação básica de DataFrames do Pandas e de como mascarar colunas de identificação.

# I CARREGAMENTO DOS PACOTES NECESSÁRIOS

In [3]:
import os
import pandas as pd
import numpy as np
import hashlib      # para mascarar identificação

from pathlib import Path

# II DIRETÓRIOS

In [4]:
os.chdir("/content/drive/MyDrive/Ensino/Ciência de Dados e Sociedade/Jorge")
root_dir   = Path(os.getcwd())
input_dir  = Path(root_dir, "Data/Input")
output_dir = Path(root_dir, "Data/Output")
output_dir

PosixPath('/content/drive/MyDrive/Ensino/Ciência de Dados e Sociedade/Jorge/Data/Output')

# Exercício 1: Manipulação Básica

In [9]:
# Tipos
types = {"CPF_fake": "str",
         "Salário": "float32",
         "Encargos": "int16"}

# Leitura
pagamentos = pd.read_table(Path(input_dir, "Folha_pagamento_comma.txt"), sep = ",",
                           index_col = False, dtype = types)
pagamentos.head(5)

,Nome,CPF_fake,Salário,Encargos
0,AÉLIO,908777362057,22771.88,634
1,ALDA,324268288911,20637.56,538
2,ALEXANDRE,766148980149,27527.34,330
3,ALINE,575223597306,29147.30,641
4,ANA,900619489281,37617.30,789


In [10]:
# a)
pagamentos = pagamentos.rename(columns = {"Nome": "nome",
                                          "CPF_fake": "cpf",
                                          "Salário": "salario",
                                          "Encargos": "encargos"})
pagamentos.head(5)

,nome,cpf,salario,encargos
0,AÉLIO,908777362057,22771.88,634
1,ALDA,324268288911,20637.56,538
2,ALEXANDRE,766148980149,27527.34,330
3,ALINE,575223597306,29147.30,641
4,ANA,900619489281,37617.30,789


In [11]:
# b)
pagamentos = pagamentos.astype({"encargos": "float32"})

pagamentos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7963 entries, 0 to 7962
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   nome      7963 non-null   object 
 1   cpf       7963 non-null   int64  
 2   salario   7963 non-null   float64
 3   encargos  7963 non-null   int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 249.0+ KB


In [ ]:
# c)
pagamentos = pagamentos.assign(custo_mensal  = lambda x: x["salario"] +  x["encargos"],
                               salario_anual = lambda x: x["salario"] * 12,
                               custo_anual   = lambda x: x["custo_mensal"] *12
                               )
pagamentos.head(5)

In [ ]:
# d)
pagamentos_10_25 = pagamentos.query(" salario > 10000 & salario < 25000 ")
pagamentos_10_25.head(5)

In [ ]:
# e)
pagamentos = pagamentos.drop(['encargos'], axis = 1)
pagamentos.head(5)

In [ ]:
# f)
pagamentos = (pd.read_table(Path(input_dir, "Folha_pagamento_comma.txt"), sep = ",",
                           index_col = False, dtype = types)
              .rename(columns = {"Nome": "nome",
                                 "CPF_fake": "cpf",
                                 "Salário": "salario",
                                 "Encargos": "encargos"})
              .astype({"encargos": "float32"})
              .assign(custo_mensal  = lambda x: x["salario"] +  x["encargos"],
                      salario_anual = lambda x: x["salario"] * 12,
                      custo_anual   = lambda x: x["custo_mensal"] *12
                      )
              .drop(['encargos'], axis = 1)
             )
pagamentos.head(5)

# Exercício 2: Mascarando Identificação

In [ ]:
# a)
pagamentos_id = (pagamentos
                 .assign(nome = lambda df:
                                 df["nome"].apply(lambda x:
                                                   hashlib.md5(x.encode('UTF-8'))
                                                   .hexdigest()
                                                   )
                         )
                 )
pagamentos_id.head(5)

In [ ]:
# b)
pagamentos_masked = (pagamentos_id
                     .assign(cpf_s = lambda df: df['cpf'] + "-stark-lannister-targaryen",
                             cpf   = lambda df:
                                     df["cpf_s"].apply(lambda x:
                                                       hashlib.sha256(x.encode('UTF-8'))
                                                       .hexdigest()
                                                       )
                              )
                      .drop(["cpf_s"], axis = 1)
                      )
pagamentos_masked.head(5)

In [ ]:
pagamentos_masked.to_pickle(Path(output_dir, "pagamentos_masked.pickle.xz"),
                            protocol = 5)
pagamentos_masked.to_csv(Path(output_dir, "pagamentos_masked.csv.xz"))